<a href="https://colab.research.google.com/github/henriquebrasileiro/pr_metrics/blob/main/PR_review_Metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Prerequisites**
You will need a GitHub Personal Access Token (PAT):
1.   Go to your GitHub Token Settings.
2.   Click Generate new token (classic).
1.   Select the repo scope.
2.   Copy the token (you will paste it into the script).





In [ ]:
import requests
import re
import csv
from datetime import datetime, timedelta
from collections import defaultdict
import math
import time
import matplotlib.pyplot as plt

# --- INTERACTIVE CONFIGURATION ---
GITHUB_TOKEN = input("Enter your GITHUB_TOKEN: ").strip()
REPO_OWNER = input("Enter the REPO_OWNER (e.g., liferay-appsec): ").strip()
REPO_NAME = "liferay-portal"

def get_dates():
    print("\n--- Date Range Setup ---")
    mode = input("Choose mode: [1] Past X Months or [2] Specific Date Range: ").strip()
    now = datetime(2026, 2, 27) # Current 2026 Reference Date

    if mode == "1":
        months = input("Number of months [Default 2]: ").strip()
        count = int(months) if months.isdigit() else 2
        return now - timedelta(days=30 * count), now
    else:
        start_str = input("Start Date (YYYY-MM-DD): ").strip()
        end_str = input("End Date (YYYY-MM-DD) [Default Today]: ").strip()
        try:
            start = datetime.strptime(start_str, "%Y-%m-%d")
            end = datetime.strptime(end_str, "%Y-%m-%d") if end_str else now
            return start, end
        except ValueError:
            print("Invalid format! Defaulting to past 2 months.")
            return now - timedelta(days=60), now

START_DATE, END_DATE = get_dates()

def extract_ticket(title):
    if not title: return "NO-TICKET"
    match = re.search(r"([A-Za-z]{3,5})[\s\-]?(\d+)", title)
    if match:
        return f"{match.group(1).upper()}-{match.group(2)}"
    return "NO-TICKET"

def get_85th_percentile(data):
    if not data: return 0
    sorted_data = sorted(data)
    index = math.ceil(len(sorted_data) * 0.85) - 1
    return sorted_data[index]

def fetch_prs(states):
    url = "https://api.github.com/graphql"
    headers = {"Authorization": f"Bearer {GITHUB_TOKEN}"}
    query = """
    query($owner: String!, $name: String!, $states: [PullRequestState!]!, $cursor: String) {
      repository(owner: $owner, name: $name) {
        pullRequests(states: $states, first: 100, after: $cursor, orderBy: {field: UPDATED_AT, direction: DESC}) {
          pageInfo { hasNextPage, endCursor }
          nodes {
            number
            title
            createdAt
            closedAt
            isDraft
            author { login }
          }
        }
      }
    }
    """
    all_prs = []
    cursor = None
    print(f"--- Fetching {states} PRs... ---")

    while True:
        variables = {"owner": REPO_OWNER, "name": REPO_NAME, "states": states, "cursor": cursor}
        response = requests.post(url, json={'query': query, 'variables': variables}, headers=headers)
        if response.status_code != 200: break

        data = response.json().get('data', {}).get('repository', {}).get('pullRequests', {})
        nodes = data.get('nodes', [])
        if not nodes: break

        filtered = [n for n in nodes if n['isDraft'] is False]
        all_prs.extend(filtered)

        last_date_str = nodes[-1]['closedAt'] or nodes[-1]['createdAt']
        last_date = datetime.fromisoformat(last_date_str.replace('Z', '+00:00')).replace(tzinfo=None)

        if last_date < START_DATE: break
        if not data['pageInfo']['hasNextPage']: break
        cursor = data['pageInfo']['endCursor']
        time.sleep(0.1)
    return all_prs

def run_full_analysis():
    closed_raw = fetch_prs(["CLOSED", "MERGED"])
    open_raw = fetch_prs(["OPEN"])

    monthly_stats = defaultdict(list)
    monthly_tickets = defaultdict(lambda: defaultdict(int))
    monthly_max = defaultdict(lambda: (0.0, "N/A"))
    all_lts = []
    all_ticket_counts = defaultdict(int)

    # Filter closed PRs correctly within range
    period_closed_prs = []
    for pr in closed_raw:
        closed_at = datetime.fromisoformat(pr['closedAt'].replace('Z', '+00:00')).replace(tzinfo=None)
        if START_DATE <= closed_at <= END_DATE:
            period_closed_prs.append(pr)
            created_at = datetime.fromisoformat(pr['createdAt'].replace('Z', '+00:00')).replace(tzinfo=None)
            lt = (closed_at - created_at).total_seconds() / 86400
            month = closed_at.strftime("%Y-%m")
            ticket = extract_ticket(pr['title'])

            monthly_stats[month].append(lt)
            monthly_tickets[month][ticket] += 1
            all_lts.append(lt)
            if ticket != "NO-TICKET": all_ticket_counts[ticket] += 1
            if lt > monthly_max[month][0]: monthly_max[month] = (lt, ticket)

    # TABLE 1: RESTORED P85 PING
    print(f"\nTABLE 1: PERFORMANCE")
    header1 = f"| {'Month':<10} | {'PRs':<5} | {'Avg LT':<8} | {'P85 LT':<8} | {'P85 Ping':<8} | {'Max LT':<8} | {'Max Ticket':<12} |"
    print(header1 + "\n" + "-"*len(header1))
    for m in sorted(monthly_stats.keys(), reverse=True):
        data = monthly_stats[m]
        p85_ping = get_85th_percentile(list(monthly_tickets[m].values()))
        print(f"| {m:<10} | {len(data):<5} | {sum(data)/len(data):<8.2f} | {get_85th_percentile(data):<8.2f} | {p85_ping:<8.2f} | {monthly_max[m][0]:<8.2f} | {monthly_max[m][1]:<12} |")

    # TABLE 2: OPEN PRS
    print(f"\nTABLE 2: CURRENTLY OPEN (NON-DRAFT)")
    header2 = f"| {'Ticket ID':<15} | {'Days in Review':<15} |"
    print(header2 + "\n" + "-"*len(header2))
    open_ages = []
    for pr in open_raw:
        created = datetime.fromisoformat(pr['createdAt'].replace('Z', '+00:00')).replace(tzinfo=None)
        age = (END_DATE - created).total_seconds() / 86400
        open_ages.append(age)
        if len(open_ages) <= 10:
            print(f"| {extract_ticket(pr['title']):<15} | {age:<15.2f} |")

    # AGGREGATE SUMMARY
    p85_lt = get_85th_percentile(all_lts)
    p85_ping_total = get_85th_percentile(list(all_ticket_counts.values())) if all_ticket_counts else 1
    months_diff = max((END_DATE - START_DATE).days / 30, 1)

    print("\n" + "="*45)
    print(f"📊 SUMMARY DASHBOARD")
    print(f"🔹 Avg PRs closed /month:      {len(all_lts) / months_diff:.2f}")
    print(f"🔹 Average Lead Time:          {sum(all_lts)/len(all_lts) if all_lts else 0:.2f} days")
    print(f"🔹 P85 Lead Time:              {p85_lt:.2f} days")
    print(f"🔹 P85 Ping-Pong Ratio:        {p85_ping_total:.2f} PRs/ticket")
    print(f"🔹 P85 Total Ticket Review:    {p85_lt * p85_ping_total:.2f} days")
    print("="*45)

    # HISTOGRAM
    plt.figure(figsize=(10, 5))
    plt.hist(open_ages, bins=20, color='#3498db', edgecolor='black')
    plt.title('Open PR Age Distribution'); plt.xlabel('Days Since Created'); plt.show()

    # CSV EXPORT WITH MONTH COLUMN
    with open("github_full_export.csv", "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Number", "State", "Ticket", "Title", "Created", "Closed", "LT (Days)", "Month Closed"])
        for pr in period_closed_prs + open_raw:
            month_closed = ""
            lt = ""
            if pr['closedAt']:
                c_date = datetime.fromisoformat(pr['closedAt'].replace('Z', '+00:00')).replace(tzinfo=None)
                month_closed = c_date.strftime("%Y-%m")
                created = datetime.fromisoformat(pr['createdAt'].replace('Z', '+00:00')).replace(tzinfo=None)
                lt = round((c_date - created).total_seconds() / 86400, 2)

            writer.writerow([pr['number'], "CLOSED" if pr['closedAt'] else "OPEN", extract_ticket(pr['title']), pr['title'], pr['createdAt'], pr['closedAt'], lt, month_closed])
    print("\n[✔] Exported: github_full_export.csv")

if __name__ == "__main__":
    run_full_analysis()